In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special.orthogonal import p_roots
import scipy.sparse as sps
import scipy.sparse.linalg

In [ ]:
class IntervalTopo1D(object):
    
    interval = (-1.0, 1.0)
    h        = 2.0
    
    def calc_jacb(self, nodes):
        do_ravel = nodes.ndim==1
        if do_ravel:
            nodes = nodes.reshape((1,-1))
        
        jacb = (nodes[:,1]-nodes[:,0])/self.h
        assert np.all(jacb!=0.0)
        
        if do_ravel: return jacb.ravel()
        return jacb
    
    def calc_jacb_det(self, jacb):
        return jacb
    
    def calc_jacb_inv(self, jacb):
        return 1.0/jacb
    
    def calc_jacb_inv_det(self, jacb):
        return 1.0/jacb
    
    def get_quadrature(self, n):
        return p_roots(n)
    
    def ref_to_phys(self, nodes, ref):
        do_ravel = nodes.ndim==1
        if do_ravel:
            nodes = nodes.reshape((1,-1))
        
        a = nodes[:,0].reshape((-1,1))
        b = (nodes[:,1]-nodes[:,0])
        b = b.reshape((-1,1))
        phys = a+b*(ref+1)/self.h
        
        if do_ravel: return phys.ravel()
        return phys
    
    def phys_to_ref(self, nodes, phys):
        pass

In [ ]:
class Mesh1D(object):
    
    def __init__(self, topo, basis):
        self.topo = topo
        self.basis = basis
        
    def build_mesh(self, nodes, elem_to_node, boundary_nodes=[]):
        self.nodes = nodes
        self.elem_to_node = elem_to_node
        
        self.n_elems  = elem_to_node.shape[0]
        self.n_nodes  = len(nodes)
        self.n_dofs   = (self.basis.n_dofs-1)*self.n_elems+1
        self.jacb     = self.topo.calc_jacb(nodes[elem_to_node])
        self.jacb_det = self.topo.calc_jacb_det(self.jacb)
        self.jacb_inv = self.topo.calc_jacb_inv(self.jacb)
        self.jacb_inv_det = self.topo.calc_jacb_inv_det(self.jacb)
        
        elem_to_dof = np.zeros((self.n_elems, self.basis.n_dofs),
                               dtype=np.int)
        
        basis = self.basis
        assert basis.n_dofs_per_node==1
        node_to_dof = np.arange(self.n_nodes)
        node_dofs = basis.node_dofs.ravel()
        elem_to_dof[:,node_dofs] = node_to_dof[elem_to_node]
        
        n_center_dofs = self.n_elems*basis.n_dofs_per_center
        center_dofs = np.arange(n_center_dofs)+len(node_to_dof)
        center_dofs = center_dofs.reshape((self.n_elems,
                                           basis.n_dofs_per_center))
        elem_to_dof[:,basis.center_dofs] = center_dofs
                                          
        self.boundary_dofs = node_to_dof[boundary_nodes]
        self.elem_to_dof = elem_to_dof

In [ ]:
class LagrangeBasis1D(object):
    
    def __init__(self, order):
        self.order = order
        self.q = order+1
        self.n_dofs = order+1
        
        roots = np.linspace(-1, 1, order+1)
        basis_polys = {}
        bp = []
        bpd1 = []
        flags = np.ones(order+1).astype(np.bool)
        for i in range(order+1):
            flags[:] = True
            flags[i] = False
            r = roots[flags]
            c = np.prod(roots[i]-r)
            p = np.poly1d(r, True)/c
            bp.append(p)
            bpd1.append(p.deriv())
            
        basis_polys[0] = bp
        basis_polys[1] = bpd1
        self.basis_polys = basis_polys
        
        ids = np.arange(order+1, dtype=np.int)
        self.node_dofs = np.array([ids[0],ids[-1]],
                                  dtype=np.int)
        self.center_dofs = ids[1:-1]
        self.n_dofs_per_node = 1
        self.n_dofs_per_center = len(self.center_dofs)
        self.n_nodes_per_elem = 2
        
    def eval_ref(self, coeffs, ref, d=0):
        
        do_ravel = coeffs.ndim==1
        if do_ravel:
            coeffs = coeffs.reshape((1,-1))
        
        res = np.zeros((coeffs.shape[0], 
                        len(ref)))
        
        polys = self.basis_polys[d]    
        for i in range(self.q):
            y = polys[i](ref)
            res += coeffs[:,i].reshape((-1,1))*y
            
        if do_ravel: return res.ravel()
        return res
            

In [ ]:
order   = 3
L       = 2*np.pi
n_elems = 20

nodes = np.linspace(0, L, n_elems+1)
elem_to_node = np.zeros((n_elems, 2), dtype=np.int)
elem_to_node[:,0] = np.arange(n_elems)
elem_to_node[:,1] = np.arange(n_elems)+1

topo  = IntervalTopo1D()
basis = LagrangeBasis1D(order)
mesh  = Mesh1D(topo, basis)
mesh.build_mesh(nodes, elem_to_node, [0,-1])

cub_points, cub_weights = topo.get_quadrature(order+1)

In [ ]:
Kloc = np.zeros((basis.n_dofs, basis.n_dofs),
                dtype=np.double)
cub_vals = basis.eval_ref(np.eye(basis.n_dofs),
                          cub_points, d=1)

for i in range(basis.n_dofs):
    for j in range(basis.n_dofs):
        Kloc[i,j] = np.sum(cub_vals[i]*cub_vals[j]*cub_weights)
        
sparse_guess = 8*n_elems*basis.n_dofs
rows = np.zeros(sparse_guess, dtype=np.double)
cols = np.zeros(sparse_guess, dtype=np.double)
vals = np.zeros(sparse_guess, dtype=np.double)

ind = 0
for ielem in range(n_elems):
    Kelem = Kloc*mesh.jacb_inv_det[ielem]
    for i in range(basis.n_dofs):
        for j in range(basis.n_dofs):
            id1 = mesh.elem_to_dof[ielem, i]
            id2 = mesh.elem_to_dof[ielem, j]
            if not (id1 in mesh.boundary_dofs) or \
                   (id2 in mesh.boundary_dofs):
                rows[ind] = id1
                cols[ind] = id2
                vals[ind] = Kelem[i,j]
                ind += 1

for idof in mesh.boundary_dofs:
    rows[ind] = idof
    cols[ind] = idof
    vals[ind] = 1.0
    ind += 1

K = sps.coo_matrix((vals[:ind],(rows[:ind],cols[:ind]))).tocsr()

In [ ]:
def f(x):
    return x*x*(x-L)

def f2(x):
    return 6*x-2*L

rhs = np.zeros(mesh.n_dofs, dtype=np.double)
phys_quad_points = topo.ref_to_phys(nodes[elem_to_node],
                                    cub_points)
f2_quad = f2(phys_quad_points)
cub_vals = basis.eval_ref(np.eye(basis.n_dofs),
                          cub_points, d=0)
a = f2_quad.reshape((f2_quad.shape[0],1,-1))*cub_vals
a = a.dot(cub_weights)
a = a*mesh.jacb_det.reshape((-1,1))

for ielem in range(n_elems):
    rhs[mesh.elem_to_dof[ielem]] += a[ielem]
rhs = -rhs
rhs[mesh.boundary_dofs] = 0.0

sol = sps.linalg.spsolve(K, rhs)

In [ ]:
x_vals = np.linspace(0, L, mesh.n_elems+1)
x_vals = (x_vals[:-1]+x_vals[1:])/2.0
s = basis.eval_ref(sol[mesh.elem_to_dof], np.array([0.0])).ravel()

np.max(np.abs(s-f(x_vals)))

In [ ]:
plt.plot(x_vals, s)
plt.plot(x_vals, f(x_vals))

In [ ]:
plt.spy(K)

In [ ]:
basis  = LagrangeBasis1D(order)

coeffs = np.eye(order+1)
x_vals = np.linspace(-1,1,100)
r  = basis.eval_ref(coeffs, x_vals)
r1 = basis.eval_ref(coeffs, x_vals, d=1)

for v in r:
    plt.plot(x_vals, v)
ylims = plt.ylim()
for v in np.linspace(-1,1,len(r)):
    plt.vlines(v, *ylims, linestyle='--', color='b')

In [ ]:
for v in r1:
    plt.plot(x_vals, v)
ylims = plt.ylim()
for v in np.linspace(-1,1,len(r1)):
    plt.vlines(v, *ylims, linestyle='--', color='b')